# Closed-loop optimization

### The surrogate model can be changed

In [ ]:
# import all required packages 
import math
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# for RF
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

#for GP
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,Matern, ConstantKernel as C

# for the NN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

#install package for excel support
#!pip install xlrd --user
import xlrd

import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs

In [ ]:
#FP_columns = ['1',' 2',' 3',' 4',' 5',' 6',' 7',' 8',' 9',' 10',' 11',' 12',' 13',' 14',' 15',' 16',' 17',' 18',' 19',' 20',' 21',' 22',' 23',' 24',' 25',' 26',' 27',' 28',' 29',' 30',' 31',' 32',' 33',' 34',' 35',' 36',' 37',' 38',' 39',' 40',' 41',' 42',' 43',' 44',' 45',' 46',' 47',' 48',' 49',' 50',' 51',' 52',' 53',' 54',' 55',' 56',' 57',' 58',' 59',' 60',' 61',' 62',' 63',' 64',' 65',' 66',' 67',' 68',' 69',' 70',' 71',' 72',' 73',' 74',' 75',' 76',' 77',' 78',' 79',' 80',' 81',' 82',' 83',' 84',' 85',' 86',' 87',' 88',' 89',' 90',' 91',' 92',' 93',' 94',' 95',' 96',' 97',' 98',' 99',' 100',' 101',' 102',' 103',' 104',' 105',' 106',' 107',' 108',' 109',' 110',' 111',' 112',' 113',' 114',' 115',' 116',' 117',' 118',' 119',' 120',' 121',' 122',' 123',' 124',' 125',' 126',' 127',' 128',' 129',' 130',' 131',' 132',' 133',' 134',' 135',' 136',' 137',' 138',' 139',' 140',' 141',' 142',' 143',' 144',' 145',' 146',' 147',' 148',' 149',' 150',' 151',' 152',' 153',' 154',' 155',' 156',' 157',' 158',' 159',' 160',' 161',' 162',' 163',' 164',' 165',' 166',' 167',' 168',' 169',' 170',' 171',' 172',' 173',' 174',' 175',' 176',' 177',' 178',' 179',' 180',' 181',' 182',' 183',' 184',' 185',' 186',' 187',' 188',' 189',' 190',' 191',' 192',' 193',' 194',' 195',' 196',' 197',' 198',' 199',' 200',' 201',' 202',' 203',' 204',' 205',' 206',' 207',' 208',' 209',' 210',' 211',' 212',' 213',' 214',' 215',' 216',' 217',' 218',' 219',' 220',' 221',' 222',' 223',' 224',' 225',' 226',' 227',' 228',' 229',' 230',' 231',' 232',' 233',' 234',' 235',' 236',' 237',' 238',' 239',' 240',' 241',' 242',' 243',' 244',' 245',' 246',' 247',' 248',' 249',' 250',' 251',' 252',' 253',' 254',' 255',' 256',' 257',' 258',' 259',' 260',' 261',' 262',' 263',' 264',' 265',' 266',' 267',' 268',' 269',' 270',' 271',' 272',' 273',' 274',' 275',' 276',' 277',' 278',' 279',' 280',' 281',' 282',' 283',' 284',' 285',' 286',' 287',' 288',' 289',' 290',' 291',' 292',' 293',' 294',' 295',' 296',' 297',' 298',' 299',' 300',' 301',' 302',' 303',' 304',' 305',' 306',' 307',' 308',' 309',' 310',' 311',' 312',' 313',' 314',' 315',' 316',' 317',' 318',' 319',' 320',' 321',' 322',' 323',' 324',' 325',' 326',' 327',' 328',' 329',' 330',' 331',' 332',' 333',' 334',' 335',' 336',' 337',' 338',' 339',' 340',' 341',' 342',' 343',' 344',' 345',' 346',' 347',' 348',' 349',' 350',' 351',' 352',' 353',' 354',' 355',' 356',' 357',' 358',' 359',' 360',' 361',' 362',' 363',' 364',' 365',' 366',' 367',' 368',' 369',' 370',' 371',' 372',' 373',' 374',' 375',' 376',' 377',' 378',' 379',' 380',' 381',' 382',' 383',' 384',' 385',' 386',' 387',' 388',' 389',' 390',' 391',' 392',' 393',' 394',' 395',' 396',' 397',' 398',' 399',' 400',' 401',' 402',' 403',' 404',' 405',' 406',' 407',' 408',' 409',' 410',' 411',' 412',' 413',' 414',' 415',' 416',' 417',' 418',' 419',' 420',' 421',' 422',' 423',' 424',' 425',' 426',' 427',' 428',' 429',' 430',' 431',' 432',' 433',' 434',' 435',' 436',' 437',' 438',' 439',' 440',' 441',' 442',' 443',' 444',' 445',' 446',' 447',' 448',' 449',' 450',' 451',' 452',' 453',' 454',' 455',' 456',' 457',' 458',' 459',' 460',' 461',' 462',' 463',' 464',' 465',' 466',' 467',' 468',' 469',' 470',' 471',' 472',' 473',' 474',' 475',' 476',' 477',' 478',' 479',' 480',' 481',' 482',' 483',' 484',' 485',' 486',' 487',' 488',' 489',' 490',' 491',' 492',' 493',' 494',' 495',' 496',' 497',' 498',' 499',' 500',' 501',' 502',' 503',' 504',' 505',' 506',' 507',' 508',' 509',' 510',' 511',' 512',' 513',' 514',' 515',' 516',' 517',' 518',' 519',' 520',' 521',' 522',' 523',' 524',' 525',' 526',' 527',' 528',' 529',' 530',' 531',' 532',' 533',' 534',' 535',' 536',' 537',' 538',' 539',' 540',' 541',' 542',' 543',' 544',' 545',' 546',' 547',' 548',' 549',' 550',' 551',' 552',' 553',' 554',' 555',' 556',' 557',' 558',' 559',' 560',' 561',' 562',' 563',' 564',' 565',' 566',' 567',' 568',' 569',' 570',' 571',' 572',' 573',' 574',' 575',' 576',' 577',' 578',' 579',' 580',' 581',' 582',' 583',' 584',' 585',' 586',' 587',' 588',' 589',' 590',' 591',' 592',' 593',' 594',' 595',' 596',' 597',' 598',' 599',' 600',' 601',' 602',' 603',' 604',' 605',' 606',' 607',' 608',' 609',' 610',' 611',' 612',' 613',' 614',' 615',' 616',' 617',' 618',' 619',' 620',' 621',' 622',' 623',' 624',' 625',' 626',' 627',' 628',' 629',' 630',' 631',' 632',' 633',' 634',' 635',' 636',' 637',' 638',' 639',' 640',' 641',' 642',' 643',' 644',' 645',' 646',' 647',' 648',' 649',' 650',' 651',' 652',' 653',' 654',' 655',' 656',' 657',' 658',' 659',' 660',' 661',' 662',' 663',' 664',' 665',' 666',' 667',' 668',' 669',' 670',' 671',' 672',' 673',' 674',' 675',' 676',' 677',' 678',' 679',' 680',' 681',' 682',' 683',' 684',' 685',' 686',' 687',' 688',' 689',' 690',' 691',' 692',' 693',' 694',' 695',' 696',' 697',' 698',' 699',' 700',' 701',' 702',' 703',' 704',' 705',' 706',' 707',' 708',' 709',' 710',' 711',' 712',' 713',' 714',' 715',' 716',' 717',' 718',' 719',' 720',' 721',' 722',' 723',' 724',' 725',' 726',' 727',' 728',' 729',' 730',' 731',' 732',' 733',' 734',' 735',' 736',' 737',' 738',' 739',' 740',' 741',' 742',' 743',' 744',' 745',' 746',' 747',' 748',' 749',' 750',' 751',' 752',' 753',' 754',' 755',' 756',' 757',' 758',' 759',' 760',' 761',' 762',' 763',' 764',' 765',' 766',' 767',' 768',' 769',' 770',' 771',' 772',' 773',' 774',' 775',' 776',' 777',' 778',' 779',' 780',' 781',' 782',' 783',' 784',' 785',' 786',' 787',' 788',' 789',' 790',' 791',' 792',' 793',' 794',' 795',' 796',' 797',' 798',' 799',' 800',' 801',' 802',' 803',' 804',' 805',' 806',' 807',' 808',' 809',' 810',' 811',' 812',' 813',' 814',' 815',' 816',' 817',' 818',' 819',' 820',' 821',' 822',' 823',' 824',' 825',' 826',' 827',' 828',' 829',' 830',' 831',' 832',' 833',' 834',' 835',' 836',' 837',' 838',' 839',' 840',' 841',' 842',' 843',' 844',' 845',' 846',' 847',' 848',' 849',' 850',' 851',' 852',' 853',' 854',' 855',' 856',' 857',' 858',' 859',' 860',' 861',' 862',' 863',' 864',' 865',' 866',' 867',' 868',' 869',' 870',' 871',' 872',' 873',' 874',' 875',' 876',' 877',' 878',' 879',' 880',' 881',' 882',' 883',' 884',' 885',' 886',' 887',' 888',' 889',' 890',' 891',' 892',' 893',' 894',' 895',' 896',' 897',' 898',' 899',' 900',' 901',' 902',' 903',' 904',' 905',' 906',' 907',' 908',' 909',' 910',' 911',' 912',' 913',' 914',' 915',' 916',' 917',' 918',' 919',' 920',' 921',' 922',' 923',' 924',' 925',' 926',' 927',' 928',' 929',' 930',' 931',' 932',' 933',' 934',' 935',' 936',' 937',' 938',' 939',' 940',' 941',' 942',' 943',' 944',' 945',' 946',' 947',' 948',' 949',' 950',' 951',' 952',' 953',' 954',' 955',' 956',' 957',' 958',' 959',' 960',' 961',' 962',' 963',' 964',' 965',' 966',' 967',' 968',' 969',' 970',' 971',' 972',' 973',' 974',' 975',' 976',' 977',' 978',' 979',' 980',' 981',' 982',' 983',' 984',' 985',' 986',' 987',' 988',' 989',' 990',' 991',' 992',' 993',' 994',' 995',' 996',' 997',' 998',' 999',' 1000',' 1001',' 1002',' 1003',' 1004',' 1005',' 1006',' 1007',' 1008',' 1009',' 1010',' 1011',' 1012',' 1013',' 1014',' 1015',' 1016',' 1017',' 1018',' 1019',' 1020',' 1021',' 1022',' 1023',' 1024',' 1025',' 1026',' 1027',' 1028',' 1029',' 1030',' 1031',' 1032',' 1033',' 1034',' 1035',' 1036',' 1037',' 1038',' 1039',' 1040',' 1041',' 1042',' 1043',' 1044',' 1045',' 1046',' 1047',' 1048',' 1049',' 1050',' 1051',' 1052',' 1053',' 1054',' 1055',' 1056',' 1057',' 1058',' 1059',' 1060',' 1061',' 1062',' 1063',' 1064',' 1065',' 1066',' 1067',' 1068',' 1069',' 1070',' 1071',' 1072',' 1073',' 1074',' 1075',' 1076',' 1077',' 1078',' 1079',' 1080',' 1081',' 1082',' 1083',' 1084',' 1085',' 1086',' 1087',' 1088',' 1089',' 1090',' 1091',' 1092',' 1093',' 1094',' 1095',' 1096',' 1097',' 1098',' 1099',' 1100',' 1101',' 1102',' 1103',' 1104',' 1105',' 1106',' 1107',' 1108',' 1109',' 1110',' 1111',' 1112',' 1113',' 1114',' 1115',' 1116',' 1117',' 1118',' 1119',' 1120',' 1121',' 1122',' 1123',' 1124',' 1125',' 1126',' 1127',' 1128',' 1129',' 1130',' 1131',' 1132',' 1133',' 1134',' 1135',' 1136',' 1137',' 1138',' 1139',' 1140',' 1141',' 1142',' 1143',' 1144',' 1145',' 1146',' 1147',' 1148',' 1149',' 1150',' 1151',' 1152',' 1153',' 1154',' 1155',' 1156',' 1157',' 1158',' 1159',' 1160',' 1161',' 1162',' 1163',' 1164',' 1165',' 1166',' 1167',' 1168',' 1169',' 1170',' 1171',' 1172',' 1173',' 1174',' 1175',' 1176',' 1177',' 1178',' 1179',' 1180',' 1181',' 1182',' 1183',' 1184',' 1185',' 1186',' 1187',' 1188',' 1189',' 1190',' 1191',' 1192',' 1193',' 1194',' 1195',' 1196',' 1197',' 1198',' 1199',' 1200',' 1201',' 1202',' 1203',' 1204',' 1205',' 1206',' 1207',' 1208',' 1209',' 1210',' 1211',' 1212',' 1213',' 1214',' 1215',' 1216',' 1217',' 1218',' 1219',' 1220',' 1221',' 1222',' 1223',' 1224',' 1225',' 1226',' 1227',' 1228',' 1229',' 1230',' 1231',' 1232',' 1233',' 1234',' 1235',' 1236',' 1237',' 1238',' 1239',' 1240',' 1241',' 1242',' 1243',' 1244',' 1245',' 1246',' 1247',' 1248',' 1249',' 1250',' 1251',' 1252',' 1253',' 1254',' 1255',' 1256',' 1257',' 1258',' 1259',' 1260',' 1261',' 1262',' 1263',' 1264',' 1265',' 1266',' 1267',' 1268',' 1269',' 1270',' 1271',' 1272',' 1273',' 1274',' 1275',' 1276',' 1277',' 1278',' 1279',' 1280',' 1281',' 1282',' 1283',' 1284',' 1285',' 1286',' 1287',' 1288',' 1289',' 1290',' 1291',' 1292',' 1293',' 1294',' 1295',' 1296',' 1297',' 1298',' 1299',' 1300',' 1301',' 1302',' 1303',' 1304',' 1305',' 1306',' 1307',' 1308',' 1309',' 1310',' 1311',' 1312',' 1313',' 1314',' 1315',' 1316',' 1317',' 1318',' 1319',' 1320',' 1321',' 1322',' 1323',' 1324',' 1325',' 1326',' 1327',' 1328',' 1329',' 1330',' 1331',' 1332',' 1333',' 1334',' 1335',' 1336',' 1337',' 1338',' 1339',' 1340',' 1341',' 1342',' 1343',' 1344',' 1345',' 1346',' 1347',' 1348',' 1349',' 1350',' 1351',' 1352',' 1353',' 1354',' 1355',' 1356',' 1357',' 1358',' 1359',' 1360',' 1361',' 1362',' 1363',' 1364',' 1365',' 1366',' 1367',' 1368',' 1369',' 1370',' 1371',' 1372',' 1373',' 1374',' 1375',' 1376',' 1377',' 1378',' 1379',' 1380',' 1381',' 1382',' 1383',' 1384',' 1385',' 1386',' 1387',' 1388',' 1389',' 1390',' 1391',' 1392',' 1393',' 1394',' 1395',' 1396',' 1397',' 1398',' 1399',' 1400',' 1401',' 1402',' 1403',' 1404',' 1405',' 1406',' 1407',' 1408',' 1409',' 1410',' 1411',' 1412',' 1413',' 1414',' 1415',' 1416',' 1417',' 1418',' 1419',' 1420',' 1421',' 1422',' 1423',' 1424',' 1425',' 1426',' 1427',' 1428',' 1429',' 1430',' 1431',' 1432',' 1433',' 1434',' 1435',' 1436',' 1437',' 1438',' 1439',' 1440',' 1441',' 1442',' 1443',' 1444',' 1445',' 1446',' 1447',' 1448',' 1449',' 1450',' 1451',' 1452',' 1453',' 1454',' 1455',' 1456',' 1457',' 1458',' 1459',' 1460',' 1461',' 1462',' 1463',' 1464',' 1465',' 1466',' 1467',' 1468',' 1469',' 1470',' 1471',' 1472',' 1473',' 1474',' 1475',' 1476',' 1477',' 1478',' 1479',' 1480',' 1481',' 1482',' 1483',' 1484',' 1485',' 1486',' 1487',' 1488',' 1489',' 1490',' 1491',' 1492',' 1493',' 1494',' 1495',' 1496',' 1497',' 1498',' 1499',' 1500',' 1501',' 1502',' 1503',' 1504',' 1505',' 1506',' 1507',' 1508',' 1509',' 1510',' 1511',' 1512',' 1513',' 1514',' 1515',' 1516',' 1517',' 1518',' 1519',' 1520',' 1521',' 1522',' 1523',' 1524',' 1525',' 1526',' 1527',' 1528',' 1529',' 1530',' 1531',' 1532',' 1533',' 1534',' 1535',' 1536',' 1537',' 1538',' 1539',' 1540',' 1541',' 1542',' 1543',' 1544',' 1545',' 1546',' 1547',' 1548',' 1549',' 1550',' 1551',' 1552',' 1553',' 1554',' 1555',' 1556',' 1557',' 1558',' 1559',' 1560',' 1561',' 1562',' 1563',' 1564',' 1565',' 1566',' 1567',' 1568',' 1569',' 1570',' 1571',' 1572',' 1573',' 1574',' 1575',' 1576',' 1577',' 1578',' 1579',' 1580',' 1581',' 1582',' 1583',' 1584',' 1585',' 1586',' 1587',' 1588',' 1589',' 1590',' 1591',' 1592',' 1593',' 1594',' 1595',' 1596',' 1597',' 1598',' 1599',' 1600',' 1601',' 1602',' 1603',' 1604',' 1605',' 1606',' 1607',' 1608',' 1609',' 1610',' 1611',' 1612',' 1613',' 1614',' 1615',' 1616',' 1617',' 1618',' 1619',' 1620',' 1621',' 1622',' 1623',' 1624',' 1625',' 1626',' 1627',' 1628',' 1629',' 1630',' 1631',' 1632',' 1633',' 1634',' 1635',' 1636',' 1637',' 1638',' 1639',' 1640',' 1641',' 1642',' 1643',' 1644',' 1645',' 1646',' 1647',' 1648',' 1649',' 1650',' 1651',' 1652',' 1653',' 1654',' 1655',' 1656',' 1657',' 1658',' 1659',' 1660',' 1661',' 1662',' 1663',' 1664',' 1665',' 1666',' 1667',' 1668',' 1669',' 1670',' 1671',' 1672',' 1673',' 1674',' 1675',' 1676',' 1677',' 1678',' 1679',' 1680',' 1681',' 1682',' 1683',' 1684',' 1685',' 1686',' 1687',' 1688',' 1689',' 1690',' 1691',' 1692',' 1693',' 1694',' 1695',' 1696',' 1697',' 1698',' 1699',' 1700',' 1701',' 1702',' 1703',' 1704',' 1705',' 1706',' 1707',' 1708',' 1709',' 1710',' 1711',' 1712',' 1713',' 1714',' 1715',' 1716',' 1717',' 1718',' 1719',' 1720',' 1721',' 1722',' 1723',' 1724',' 1725',' 1726',' 1727',' 1728',' 1729',' 1730',' 1731',' 1732',' 1733',' 1734',' 1735',' 1736',' 1737',' 1738',' 1739',' 1740',' 1741',' 1742',' 1743',' 1744',' 1745',' 1746',' 1747',' 1748',' 1749',' 1750',' 1751',' 1752',' 1753',' 1754',' 1755',' 1756',' 1757',' 1758',' 1759',' 1760',' 1761',' 1762',' 1763',' 1764',' 1765',' 1766',' 1767',' 1768',' 1769',' 1770',' 1771',' 1772',' 1773',' 1774',' 1775',' 1776',' 1777',' 1778',' 1779',' 1780',' 1781',' 1782',' 1783',' 1784',' 1785',' 1786',' 1787',' 1788',' 1789',' 1790',' 1791',' 1792',' 1793',' 1794',' 1795',' 1796',' 1797',' 1798',' 1799',' 1800',' 1801',' 1802',' 1803',' 1804',' 1805',' 1806',' 1807',' 1808',' 1809',' 1810',' 1811',' 1812',' 1813',' 1814',' 1815',' 1816',' 1817',' 1818',' 1819',' 1820',' 1821',' 1822',' 1823',' 1824',' 1825',' 1826',' 1827',' 1828',' 1829',' 1830',' 1831',' 1832',' 1833',' 1834',' 1835',' 1836',' 1837',' 1838',' 1839',' 1840',' 1841',' 1842',' 1843',' 1844',' 1845',' 1846',' 1847',' 1848',' 1849',' 1850',' 1851',' 1852',' 1853',' 1854',' 1855',' 1856',' 1857',' 1858',' 1859',' 1860',' 1861',' 1862',' 1863',' 1864',' 1865',' 1866',' 1867',' 1868',' 1869',' 1870',' 1871',' 1872',' 1873',' 1874',' 1875',' 1876',' 1877',' 1878',' 1879',' 1880',' 1881',' 1882',' 1883',' 1884',' 1885',' 1886',' 1887',' 1888',' 1889',' 1890',' 1891',' 1892',' 1893',' 1894',' 1895',' 1896',' 1897',' 1898',' 1899',' 1900',' 1901',' 1902',' 1903',' 1904',' 1905',' 1906',' 1907',' 1908',' 1909',' 1910',' 1911',' 1912',' 1913',' 1914',' 1915',' 1916',' 1917',' 1918',' 1919',' 1920',' 1921',' 1922',' 1923',' 1924',' 1925',' 1926',' 1927',' 1928',' 1929',' 1930',' 1931',' 1932',' 1933',' 1934',' 1935',' 1936',' 1937',' 1938',' 1939',' 1940',' 1941',' 1942',' 1943',' 1944',' 1945',' 1946',' 1947',' 1948',' 1949',' 1950',' 1951',' 1952',' 1953',' 1954',' 1955',' 1956',' 1957',' 1958',' 1959',' 1960',' 1961',' 1962',' 1963',' 1964',' 1965',' 1966',' 1967',' 1968',' 1969',' 1970',' 1971',' 1972',' 1973',' 1974',' 1975',' 1976',' 1977',' 1978',' 1979',' 1980',' 1981',' 1982',' 1983',' 1984',' 1985',' 1986',' 1987',' 1988',' 1989',' 1990',' 1991',' 1992',' 1993',' 1994',' 1995',' 1996',' 1997',' 1998',' 1999',' 2000',' 2001',' 2002',' 2003',' 2004',' 2005',' 2006',' 2007',' 2008',' 2009',' 2010',' 2011',' 2012',' 2013',' 2014',' 2015',' 2016',' 2017',' 2018',' 2019',' 2020',' 2021',' 2022',' 2023',' 2024',' 2025',' 2026',' 2027',' 2028',' 2029',' 2030',' 2031',' 2032',' 2033',' 2034',' 2035',' 2036',' 2037',' 2038',' 2039',' 2040',' 2041',' 2042',' 2043',' 2044',' 2045',' 2046',' 2047',' 2048',' 2049',' 2050',' 2051',' 2052',' 2053',' 2054',' 2055',' 2056',' 2057',' 2058',' 2059',' 2060',' 2061',' 2062',' 2063',' 2064',' 2065',' 2066',' 2067',' 2068',' 2069',' 2070',' 2071',' 2072',' 2073',' 2074',' 2075',' 2076',' 2077',' 2078',' 2079',' 2080',' 2081',' 2082',' 2083',' 2084',' 2085',' 2086',' 2087',' 2088',' 2089',' 2090',' 2091',' 2092',' 2093',' 2094',' 2095',' 2096',' 2097',' 2098',' 2099',' 2100',' 2101',' 2102',' 2103',' 2104',' 2105',' 2106',' 2107',' 2108',' 2109',' 2110',' 2111',' 2112',' 2113',' 2114',' 2115',' 2116',' 2117',' 2118',' 2119',' 2120',' 2121',' 2122',' 2123',' 2124',' 2125',' 2126',' 2127',' 2128',' 2129',' 2130',' 2131',' 2132',' 2133',' 2134',' 2135',' 2136',' 2137',' 2138',' 2139',' 2140',' 2141',' 2142',' 2143',' 2144',' 2145',' 2146',' 2147',' 2148',' 2149',' 2150',' 2151',' 2152',' 2153',' 2154',' 2155',' 2156',' 2157',' 2158',' 2159',' 2160',' 2161',' 2162',' 2163',' 2164',' 2165',' 2166',' 2167',' 2168',' 2169',' 2170',' 2171',' 2172',' 2173',' 2174',' 2175',' 2176',' 2177',' 2178',' 2179',' 2180',' 2181',' 2182',' 2183',' 2184',' 2185',' 2186',' 2187',' 2188',' 2189',' 2190',' 2191',' 2192',' 2193',' 2194',' 2195',' 2196',' 2197',' 2198',' 2199',' 2200',' 2201',' 2202',' 2203',' 2204',' 2205',' 2206',' 2207',' 2208',' 2209',' 2210',' 2211',' 2212',' 2213',' 2214',' 2215',' 2216',' 2217',' 2218',' 2219',' 2220',' 2221',' 2222',' 2223',' 2224',' 2225',' 2226',' 2227',' 2228',' 2229',' 2230',' 2231',' 2232',' 2233',' 2234',' 2235',' 2236',' 2237',' 2238',' 2239',' 2240',' 2241',' 2242',' 2243',' 2244',' 2245',' 2246',' 2247',' 2248',' 2249',' 2250',' 2251',' 2252',' 2253',' 2254',' 2255',' 2256',' 2257',' 2258',' 2259',' 2260',' 2261',' 2262',' 2263',' 2264',' 2265',' 2266',' 2267',' 2268',' 2269',' 2270',' 2271',' 2272',' 2273',' 2274',' 2275',' 2276',' 2277',' 2278',' 2279',' 2280',' 2281',' 2282',' 2283',' 2284',' 2285',' 2286',' 2287',' 2288',' 2289',' 2290',' 2291',' 2292',' 2293',' 2294',' 2295',' 2296',' 2297',' 2298',' 2299',' 2300',' 2301',' 2302',' 2303',' 2304',' 2305',' 2306',' 2307',' 2308',' 2309',' 2310',' 2311',' 2312',' 2313',' 2314',' 2315',' 2316',' 2317',' 2318',' 2319',' 2320',' 2321',' 2322',' 2323',' 2324',' 2325',' 2326',' 2327',' 2328',' 2329',' 2330',' 2331',' 2332',' 2333',' 2334',' 2335',' 2336',' 2337',' 2338',' 2339',' 2340',' 2341',' 2342',' 2343',' 2344',' 2345',' 2346',' 2347',' 2348',' 2349',' 2350',' 2351',' 2352',' 2353',' 2354',' 2355',' 2356',' 2357',' 2358',' 2359',' 2360',' 2361',' 2362',' 2363',' 2364',' 2365',' 2366',' 2367',' 2368',' 2369',' 2370',' 2371',' 2372',' 2373',' 2374',' 2375',' 2376',' 2377',' 2378',' 2379',' 2380',' 2381',' 2382',' 2383',' 2384',' 2385',' 2386',' 2387',' 2388',' 2389',' 2390',' 2391',' 2392',' 2393',' 2394',' 2395',' 2396',' 2397',' 2398',' 2399',' 2400',' 2401',' 2402',' 2403',' 2404',' 2405',' 2406',' 2407',' 2408',' 2409',' 2410',' 2411',' 2412',' 2413',' 2414',' 2415',' 2416',' 2417',' 2418',' 2419',' 2420',' 2421',' 2422',' 2423',' 2424',' 2425',' 2426',' 2427',' 2428',' 2429',' 2430',' 2431',' 2432',' 2433',' 2434',' 2435',' 2436',' 2437',' 2438',' 2439',' 2440',' 2441',' 2442',' 2443',' 2444',' 2445',' 2446',' 2447',' 2448',' 2449',' 2450',' 2451',' 2452',' 2453',' 2454',' 2455',' 2456',' 2457',' 2458',' 2459',' 2460',' 2461',' 2462',' 2463',' 2464',' 2465',' 2466',' 2467',' 2468',' 2469',' 2470',' 2471',' 2472',' 2473',' 2474',' 2475',' 2476',' 2477',' 2478',' 2479',' 2480',' 2481',' 2482',' 2483',' 2484',' 2485',' 2486',' 2487',' 2488',' 2489',' 2490',' 2491',' 2492',' 2493',' 2494',' 2495',' 2496',' 2497',' 2498',' 2499',' 2500',' 2501',' 2502',' 2503',' 2504',' 2505',' 2506',' 2507',' 2508',' 2509',' 2510',' 2511',' 2512',' 2513',' 2514',' 2515',' 2516',' 2517',' 2518',' 2519',' 2520',' 2521',' 2522',' 2523',' 2524',' 2525',' 2526',' 2527',' 2528',' 2529',' 2530',' 2531',' 2532',' 2533',' 2534',' 2535',' 2536',' 2537',' 2538',' 2539',' 2540',' 2541',' 2542',' 2543',' 2544',' 2545',' 2546',' 2547',' 2548',' 2549',' 2550',' 2551',' 2552',' 2553',' 2554',' 2555',' 2556',' 2557',' 2558',' 2559',' 2560',' 2561',' 2562',' 2563',' 2564',' 2565',' 2566',' 2567',' 2568',' 2569',' 2570',' 2571',' 2572',' 2573',' 2574',' 2575',' 2576',' 2577',' 2578',' 2579',' 2580',' 2581',' 2582',' 2583',' 2584',' 2585',' 2586',' 2587',' 2588',' 2589',' 2590',' 2591',' 2592',' 2593',' 2594',' 2595',' 2596',' 2597',' 2598',' 2599',' 2600',' 2601',' 2602',' 2603',' 2604',' 2605',' 2606',' 2607',' 2608',' 2609',' 2610',' 2611',' 2612',' 2613',' 2614',' 2615',' 2616',' 2617',' 2618',' 2619',' 2620',' 2621',' 2622',' 2623',' 2624',' 2625',' 2626',' 2627',' 2628',' 2629',' 2630',' 2631',' 2632',' 2633',' 2634',' 2635',' 2636',' 2637',' 2638',' 2639',' 2640',' 2641',' 2642',' 2643',' 2644',' 2645',' 2646',' 2647',' 2648',' 2649',' 2650',' 2651',' 2652',' 2653',' 2654',' 2655',' 2656',' 2657',' 2658',' 2659',' 2660',' 2661',' 2662',' 2663',' 2664',' 2665',' 2666',' 2667',' 2668',' 2669',' 2670',' 2671',' 2672',' 2673',' 2674',' 2675',' 2676',' 2677',' 2678',' 2679',' 2680',' 2681',' 2682',' 2683',' 2684',' 2685',' 2686',' 2687',' 2688',' 2689',' 2690',' 2691',' 2692',' 2693',' 2694',' 2695',' 2696',' 2697',' 2698',' 2699',' 2700',' 2701',' 2702',' 2703',' 2704',' 2705',' 2706',' 2707',' 2708',' 2709',' 2710',' 2711',' 2712',' 2713',' 2714',' 2715',' 2716',' 2717',' 2718',' 2719',' 2720',' 2721',' 2722',' 2723',' 2724',' 2725',' 2726',' 2727',' 2728',' 2729',' 2730',' 2731',' 2732',' 2733',' 2734',' 2735',' 2736',' 2737',' 2738',' 2739',' 2740',' 2741',' 2742',' 2743',' 2744',' 2745',' 2746',' 2747',' 2748',' 2749',' 2750',' 2751',' 2752',' 2753',' 2754',' 2755',' 2756',' 2757',' 2758',' 2759',' 2760',' 2761',' 2762',' 2763',' 2764',' 2765',' 2766',' 2767',' 2768',' 2769',' 2770',' 2771',' 2772',' 2773',' 2774',' 2775',' 2776',' 2777',' 2778',' 2779',' 2780',' 2781',' 2782',' 2783',' 2784',' 2785',' 2786',' 2787',' 2788',' 2789',' 2790',' 2791',' 2792',' 2793',' 2794',' 2795',' 2796',' 2797',' 2798',' 2799',' 2800',' 2801',' 2802',' 2803',' 2804',' 2805',' 2806',' 2807',' 2808',' 2809',' 2810',' 2811',' 2812',' 2813',' 2814',' 2815',' 2816',' 2817',' 2818',' 2819',' 2820',' 2821',' 2822',' 2823',' 2824',' 2825',' 2826',' 2827',' 2828',' 2829',' 2830',' 2831',' 2832',' 2833',' 2834',' 2835',' 2836',' 2837',' 2838',' 2839',' 2840',' 2841',' 2842',' 2843',' 2844',' 2845',' 2846',' 2847',' 2848',' 2849',' 2850',' 2851',' 2852',' 2853',' 2854',' 2855',' 2856',' 2857',' 2858',' 2859',' 2860',' 2861',' 2862',' 2863',' 2864',' 2865',' 2866',' 2867',' 2868',' 2869',' 2870',' 2871',' 2872',' 2873',' 2874',' 2875',' 2876',' 2877',' 2878',' 2879',' 2880',' 2881',' 2882',' 2883',' 2884',' 2885',' 2886',' 2887',' 2888',' 2889',' 2890',' 2891',' 2892',' 2893',' 2894',' 2895',' 2896',' 2897',' 2898',' 2899',' 2900',' 2901',' 2902',' 2903',' 2904',' 2905',' 2906',' 2907',' 2908',' 2909',' 2910',' 2911',' 2912',' 2913',' 2914',' 2915',' 2916',' 2917',' 2918',' 2919',' 2920',' 2921',' 2922',' 2923',' 2924',' 2925',' 2926',' 2927',' 2928',' 2929',' 2930',' 2931',' 2932',' 2933',' 2934',' 2935',' 2936',' 2937',' 2938',' 2939',' 2940',' 2941',' 2942',' 2943',' 2944',' 2945',' 2946',' 2947',' 2948',' 2949',' 2950',' 2951',' 2952',' 2953',' 2954',' 2955',' 2956',' 2957',' 2958',' 2959',' 2960',' 2961',' 2962',' 2963',' 2964',' 2965',' 2966',' 2967',' 2968',' 2969',' 2970',' 2971',' 2972',' 2973',' 2974',' 2975',' 2976',' 2977',' 2978',' 2979',' 2980',' 2981',' 2982',' 2983',' 2984',' 2985',' 2986',' 2987',' 2988',' 2989',' 2990',' 2991',' 2992',' 2993',' 2994',' 2995',' 2996',' 2997',' 2998',' 2999',' 3000',' 3001',' 3002',' 3003',' 3004',' 3005',' 3006',' 3007',' 3008',' 3009',' 3010',' 3011',' 3012',' 3013',' 3014',' 3015',' 3016',' 3017',' 3018',' 3019',' 3020',' 3021',' 3022',' 3023',' 3024',' 3025',' 3026',' 3027',' 3028',' 3029',' 3030',' 3031',' 3032',' 3033',' 3034',' 3035',' 3036',' 3037',' 3038',' 3039',' 3040',' 3041',' 3042',' 3043',' 3044',' 3045',' 3046',' 3047',' 3048',' 3049',' 3050',' 3051',' 3052',' 3053',' 3054',' 3055',' 3056',' 3057',' 3058',' 3059',' 3060',' 3061',' 3062',' 3063',' 3064',' 3065',' 3066',' 3067',' 3068',' 3069',' 3070',' 3071',' 3072']

In [ ]:
# Get the inputs and the outputs ready

full_dataset = data_2_sterimol
full_dataset = np.append(full_dataset,dft_data,axis=1)
full_dataset = np.append(full_dataset,fp_pca_data,axis=1)
full_dataset = np.append(full_dataset,dataset_one_hot,axis=1)
full_dataset = np.append(full_dataset,NBO_data,axis=1)
full_dataset = np.append(full_dataset,chelpg_data,axis=1)
full_dataset = np.append(full_dataset,vol_bur,axis=1)
full_dataset = np.append(full_dataset,n_proton,axis=1)
full_dataset = np.append(full_dataset,yield_numerical,axis=1)

feature_column_names = ['N_sterimol_l','N_sterimol_b1','N_sterimol_b5','R_sterimol_l','R_sterimol_b1','R_sterimol_b5',
                       'homo', 'lumo',
                       'ligands_PCA_1',' ligands_PCA_2',' ligands_PCA_3','ligands_PCA_4','ligands_PCA_5','ligands_PCA_6','ligands_PCA_7','ligands_PCA_8','ligands_PCA_9','ligands_PCA_10','ligands_PCA_11','ligands_PCA_12','ligands_PCA_13','ligands_PCA_14','ligands_PCA_15','ligands_PCA_16','ligands_PCA_17','ligands_PCA_18','ligands_PCA_19','ligands_PCA_20','ligands_PCA_21','ligands_PCA_22','ligands_PCA_23','ligands_PCA_24','ligands_PCA_25','ligands_PCA_26','ligands_PCA_27','ligands_PCA_28','ligands_PCA_29','ligands_PCA_30',
                       'precat_PCA_1',' precat_PCA_2',' precat_PCA_3','precat_PCA_4','precat_PCA_5','precat_PCA_6','precat_PCA_7','precat_PCA_8','precat_PCA_9','precat_PCA_10','precat_PCA_11','precat_PCA_12','precat_PCA_13','precat_PCA_14','precat_PCA_15','precat_PCA_16','precat_PCA_17','precat_PCA_18','precat_PCA_19','precat_PCA_20','precat_PCA_21','precat_PCA_22','precat_PCA_23','precat_PCA_24','precat_PCA_25','precat_PCA_26','precat_PCA_27','precat_PCA_28','precat_PCA_29','precat_PCA_30',
                       'boronic_PCA_1',' boronic_PCA_2',' boronic_PCA_3','boronic_PCA_4','boronic_PCA_5','boronic_PCA_6','boronic_PCA_7','boronic_PCA_8','boronic_PCA_9','boronic_PCA_10','boronic_PCA_11','boronic_PCA_12','boronic_PCA_13','boronic_PCA_14','boronic_PCA_15','boronic_PCA_16','boronic_PCA_17','boronic_PCA_18','boronic_PCA_19','boronic_PCA_20','boronic_PCA_21','boronic_PCA_22','boronic_PCA_23','boronic_PCA_24','boronic_PCA_25','boronic_PCA_26','boronic_PCA_27','boronic_PCA_28','boronic_PCA_29','boronic_PCA_30',
                       'OHE_lig_1',' OHE_lig_2',' OHE_lig_3','OHE_lig_4','OHE_lig_5','OHE_lig_6','OHE_lig_7','OHE_lig_9','OHE_lig_10','OHE_lig_11','OHE_lig_12','OHE_lig_13','OHE_lig_14','OHE_lig_15','OHE_lig_16','OHE_lig_17','OHE_lig_18','OHE_lig_19','OHE_lig_20','OHE_lig_21','OHE_lig_22','OHE_lig_23','OHE_lig_24','OHE_lig_25','OHE_lig_26','OHE_lig_27','OHE_lig_28','OHE_lig_29','OHE_lig_30',' OHE_lig_31',' OHE_lig_32',
                       'OHE_precat_1','OHE_precat_2','OHE_precat_3',
                       'OHE_bor_1','OHE_bor_2',
                       'NBO_N','NBO_OR','NBO_O','NBO_AcO',
                       'chelpg_N','chelpg_OR','chelpg_O','chelpg_AcO',
                       'vol_bur',
                       'n_proton',
                       'Yield']

full_dataset_input = pd.DataFrame(full_dataset)
full_dataset_input.columns = feature_column_names

print('Shape of the featurized dataset:',full_dataset_input.shape)

In [ ]:
# To allow for fair comparison across different models when conducting 10 repetitions, the data used for initialization was set a priori for various initialization strategies

# using 5 datapoints for initialization
#random_initialization = [183, 127, 107, 118, 48, 16, 71, 89, 90, 64, 42, 70, 29, 4, 42, 139, 89, 51, 167]
#random_initialization = [44, 92, 179, 151, 129, 170, 110, 177, 133, 165, 151, 162, 102, 3, 78, 99, 70, 37, 55]
#random_initialization = [161, 23, 86, 39, 136, 114, 87, 177, 3, 2, 158, 70, 156, 130, 85, 106, 155, 114, 142]
#random_initialization = [140, 47, 117, 63, 33, 73, 41, 87, 173, 67, 80, 93, 182, 112, 1, 127, 134, 20, 31]
#random_initialization = [84, 5, 4, 90, 86, 165, 68, 85, 29, 184, 113, 43, 178, 63, 25, 12, 37, 180, 93]
#random_initialization = [71, 70, 179, 2, 182, 19, 182, 70, 184, 170, 18, 132, 74, 107, 171, 90, 116, 123, 162]
#random_initialization = [76, 184, 123, 40, 24, 83, 48, 98, 63, 14, 124, 123, 51, 128, 16, 137, 1, 74, 44]
#random_initialization = [77, 41, 131, 114, 50, 65, 14, 56, 95, 71, 167, 137, 57, 135, 4, 165, 2, 105, 89]
#random_initialization = [165, 81, 147, 19, 117, 62, 80, 124, 139, 7, 28, 107, 149, 55, 78, 44, 37, 135, 32]
#random_initialization = [123, 52, 43, 101, 43, 98, 137, 93, 87, 85, 10, 152, 131, 53, 49, 76, 100, 44, 57]

#using 10 datapoints for initialization
#random_initialization = [162, 129,  89, 40,124,  16, 165, 100, 177, 108, 138, 104, 103,  79, 152, 137, 112,44, 124, 175, 164, 149,   9,  36]
#random_initialization = [161,  98, 147, 158, 118, 180, 184,  93,  57,  31, 136, 103, 151, 112, 162,  57,   1,  74,  62, 112,   7,  55,  99,   9]
#random_initialization = [155,  23, 169,   2,  80,  56, 179, 139,  73,  42,  41,  83, 168, 64, 155,  23,  54, 171,  42, 166, 122, 106,   8, 147]
#random_initialization = [ 68,  54, 163, 175,  62, 108,  85,  24, 180, 116, 123,  25, 149, 87,  79,   1,  47, 104, 113,  69,   0, 175,  85,  79]
#random_initialization = [ 52, 127,  89,  14,  64,   3,  41,  55,  84, 165,   0,  28,  30, 20, 124,  72,  52,  68,   9,  49, 184, 139, 127, 138]
#random_initialization = [ 95,   9,  86,  82, 177,  85, 183, 133,  63, 155,  76,   6, 173, 71,  15, 179,  51,  28,  10, 102,  57,  73,   1, 129]
#random_initialization = [ 95, 183, 118, 110,  61,  24,  82, 143, 150,  37, 138, 166,  18, 131,  35,  34, 159,  23, 139,  68,  22,  18, 153, 120]
#random_initialization = [ 33,  82, 161,  49,  12, 116,  40,  87,  50, 164, 177,  37, 126, 166,  32, 127, 180,  68,  74, 134, 103,  35,   9,  24]
#random_initialization = [131,  78,  13,  81, 139,  30,  79,  25,  29,  73,  46,  50,  19, 60, 151, 108, 121, 134,  72,  79,  43, 161, 132,  97]
#random_initialization = [ 22,   2, 129,  68, 161,   7,  63,  24,  85,  18,  92, 162, 175, 45, 153,   0,  67,  73,  92,  69, 153,  61,  10, 149]

#print(len(random_initialization))
# using 15 datapoints for initialization
#random_initialization = [116, 78, 65, 148, 108, 175, 149, 27, 81, 38, 119, 26, 47, 136, 96, 87, 47, 2, 62, 17, 159, 91, 37, 114, 70, 24, 168, 11, 50]
#random_initialization = [59, 167, 175, 24, 116, 111, 100, 113, 32, 167, 52, 4, 140, 131, 87, 109, 103, 145, 70, 0, 76, 149, 29, 46, 39, 111, 19, 41, 41]
#random_initialization = [85, 14, 3, 111, 66, 59, 25, 170, 39, 93, 61, 138, 93, 136, 74, 1, 119, 68, 79, 108, 137, 25, 77, 156, 164, 118, 131, 164, 49]
#random_initialization = [118, 99, 134, 56, 92, 80, 157, 183, 70, 133, 120, 105, 102, 100, 56, 79, 112, 106, 37, 68, 118, 27, 0, 162, 97, 54, 130, 163, 61]
#random_initialization = [10, 6, 6, 31, 35, 133, 142, 104, 27, 0, 72, 133, 147, 53, 137, 148, 23, 61, 137, 51, 47, 165, 8, 77, 16, 81, 31, 22, 171]
#random_initialization = [92, 77, 183, 142, 37, 109, 180, 25, 82, 65, 134, 175, 79, 175, 113, 120, 179, 97, 158, 29, 181, 104, 67, 137, 30, 132, 73, 137, 30]
#random_initialization = [4, 11, 119, 77, 10, 73, 99, 9, 54, 56, 159, 140, 51, 124, 20, 66, 38, 159, 143, 106, 170, 87, 86, 50, 142, 43, 58, 136, 91]
#random_initialization = [129, 59, 49, 11, 98, 138, 174, 12, 87, 131, 146, 127, 8, 26, 102, 41, 8, 118, 170, 148, 177, 69, 13, 143, 54, 123, 172, 182, 73]
#random_initialization = [83, 182, 181, 45, 150, 20, 32, 75, 149, 92, 67, 71, 155, 85, 61, 15, 60, 93, 37, 28, 129, 47, 125, 49, 80, 26, 111, 44, 115]
#random_initialization = [93, 3, 82, 14, 89, 29, 21, 104, 155, 105, 0, 87, 106, 147, 20, 136, 82, 136, 159, 8, 96, 124, 24, 50, 25, 121, 43, 8, 44]

#using 20 datapoints for initialization
#random_initialization = [86, 13, 25, 149, 125, 141, 96, 39, 138, 59, 64, 104, 82, 133, 88, 27, 52, 60, 159, 105, 122, 176, 24, 152, 134, 68, 52, 102, 173, 28, 102, 63, 50, 18]
#random_initialization = [150, 103, 52, 7, 3, 72, 48, 115, 7, 67, 97, 184, 164, 26, 55, 173, 3, 70, 74, 114, 26, 89, 84, 158, 155, 97, 99, 174, 136, 111, 16, 72, 134, 20]
#random_initialization = [179, 14, 101, 94, 14, 161, 147, 118, 66, 155, 80, 40, 154, 154, 28, 46, 155, 31, 125, 5, 15, 13, 53, 131, 131, 12, 71, 112, 114, 114, 140, 141, 174, 91]
#random_initialization = [91, 92, 136, 131, 177, 53, 58, 48, 14, 41, 51, 102, 125, 149, 28, 179, 117, 175, 174, 183, 111, 35, 137, 143, 24, 118, 103, 68, 70, 0, 175, 183, 155, 109]
#random_initialization = [152, 177, 94, 112, 98, 151, 106, 106, 11, 81, 36, 58, 54, 115, 105, 38, 74, 31, 135, 38, 177, 63, 120, 167, 156, 142, 63, 176, 180, 40, 61, 15, 114, 30]
#random_initialization = [77, 112, 90, 28, 76, 50, 48, 90, 131, 103, 34, 28, 32, 69, 22, 118, 129, 19, 115, 22, 117, 116, 148, 108, 99, 90, 128, 174, 5, 105, 70, 172, 40, 94]
#random_initialization = [139, 131, 154, 5, 13, 12, 124, 150, 155, 7, 24, 173, 183, 142, 86, 28, 148, 53, 6, 107, 182, 1, 9, 143, 55, 9, 166, 85, 156, 61, 16, 16, 26, 3]
#random_initialization = [182, 62, 184, 88, 153, 20, 100, 169, 171, 66, 183, 89, 107, 18, 154, 68, 26, 80, 95, 106, 119, 23, 3, 167, 110, 79, 6, 93, 141, 7, 113, 83, 74, 136]
#random_initialization = [49, 154, 13, 64, 159, 87, 143, 127, 67, 69, 131, 181, 140, 81, 101, 53, 109, 161, 121, 155, 20, 59, 55, 121, 101, 32, 173, 120, 174, 32, 108, 106, 45, 160]
#random_initialization = [85, 174, 147, 104, 37, 173, 27, 181, 152, 127, 47, 6, 166, 156, 182, 54, 149, 97, 81, 72, 57, 9, 170, 174, 110, 171, 148, 55, 142, 25, 118, 130, 170, 105]

#local initialization
#random_initialization =  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24] 
#random_initialization =  [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48] 
#random_initialization = [49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72] 
#random_initialization = [73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96] 
#random_initialization = [97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120] 
#random_initialization = [121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144] 
#random_initialization = [145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168] 
#random_initialization = [161, 162, 163, 164, 165, 166, 167, 168,169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184]

initial_data = full_dataset_input.loc[random_initialization]


#now we need to delete the datapoints from the original dataset so that they don't appear twice
lookup_table = full_dataset_input.drop(random_initialization)

# now we can finally shuffle the dataframe
lookup_table = shuffle(lookup_table)

#### Illustration of the data for initialization in the  chemical space

In [ ]:
# Let's better get the first two principle components from the whole dataset and then take out the initial and lookup principal components
pca_illustrate = PCA(n_components=2)
fingerprints_pca_2 = pca_illustrate.fit_transform(full_dataset_input)
print(fingerprints_pca_2.shape)
fingerprints_pca_2 = np.array(fingerprints_pca_2)

#let's pull out the PC from the initial dataset
fingerprints_init_pca = fingerprints_pca_2[random_initialization]
print(fingerprints_init_pca.shape)
#print(fingerprints_init_pca)

# let's delete the initial data from the whole lookup table
fingerprints_lookupTable_pca = np.delete(fingerprints_pca_2, random_initialization, 0)
print(fingerprints_lookupTable_pca.shape)

# Let's get the plot ready
plt.title(label="Local Initialization",fontsize=16)

plt.scatter(fingerprints_lookupTable_pca[:,0], fingerprints_lookupTable_pca[:,1],label='Lookup data',color='dodgerblue')
plt.scatter(fingerprints_init_pca[10:,0],fingerprints_init_pca[10:,1] ,color='dodgerblue')
plt.scatter(fingerprints_init_pca[:10,0],fingerprints_init_pca[:10,1] ,label='Initialization data',color='red')

plt.xlabel('PC1',fontsize=14)
plt.ylabel('PC2',fontsize=14)

plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)
plt.rc('legend', fontsize=13) 

plt.legend()

print('is there something wrong, the PCA seems to no be able to separete the ')

In [ ]:
# Let's split the large dataset into the data which we will use for building the initial model

lookup_output = lookup_table['Yield']
print(lookup_output.shape)

lookup_input = lookup_table.drop(columns='Yield')
print(lookup_input.shape)


In [ ]:
dataset_shape = []

RMSE = []

lookup_input = np.array(lookup_input)

lookup_output = np.array(lookup_output)

lookup_output = lookup_output.reshape(-1,1)

pred_yield_development = []

# set the very initial train features and labels
train_features = train_features_init
train_labels = train_labels_init

current_maximum =[]

for i in range (153):
    train_x = train_features 
    train_y = train_labels
    
    # let's train the RF
    regressor = RandomForestRegressor(n_estimators=200, random_state=0)
    regressor.fit(train_x, train_y)
    test_predictions = regressor.predict(test_features_init)
    
    # let's evaluate the RF
    rmse = metrics.mean_absolute_error(test_labels_init, test_predictions)
    RMSE.append(rmse)
    
    # let's do predictions over the unexplored area and evaluate the best predictions
    lookup_predictions = regressor.predict(lookup_input)
    
    # append the current best datapoint to the table current_maximum
    current_maximum.append(np.max(train_labels))

    # evaluate the highest predicted datapoint given the whole chemical space and the trained model
    best = np.max(lookup_predictions)
    List = lookup_predictions.tolist()
    index = List.index(best)
    
    new_input = lookup_input[index].reshape(1,-1)
    new_output = lookup_output[index].reshape(1,-1)
    
    train_features = np.append(train_features,new_input,axis=0)
    train_labels = np.append(train_labels,lookup_output[index],axis=0)
    
    lookup_input = np.delete(lookup_input, [index], 0)
    lookup_output = np.delete(lookup_output, [index], 0)
    
    # append the best predicted yield to the table pre_yield_development
    pred_yield_development.append(new_output)
    
    print(i,':The current best datapoint is:',np.max(train_labels))